# Introduction to Machine Learning using Muon Tracks in CMS  


Write code to expand the pseudocode below into an investigation of [muon track data](https://github.com/QuarkNet-HEP/coding-camp/tree/main/data) in the CMS detector using Machine Learning techniques. You may find the textbook Chapter 4 of 'Hands-on Machine Learning with Scikit-Learn, Keras & TensorFlow' by O'Reilly helpful.

## Problem  
Can we use machine learning to improve our fitting techniques?  

## Plan  
Use machine learning regression techniques to fit model functions to CMS tracker data for a muon. This is based on the Muon Tracks activity with machine learning application.

## Data  
For muon track 1, pt = 25 GeV, particle is a muon with q= -1. muon_track_1.csv has 3 rows of header info to ignore for analysis as a pandas dataframe. Data include x,y coordinates of the muon's trajectory but not z. The tracker has a spatial resolution on the order of 0.5 cm due to physical size of the hardware elements.  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import linear_model
from scipy.optimize import curve_fit

In [ ]:
track = pd.read_csv('https://github.com/QuarkNet-HEP/coding-camp/raw/main/data/muon_track_1.csv', skiprows=2)
track.head()

## Analysis  

### Linear Regression

We will use the function [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) from the Scikit-Learn library to train a linear regression model. Training a model means setting its parameters so that the model best fits the training dataset using a performace measure. This is done by finding parameters in the model that minimizes the Root Mean Square Error or the Mean Square Error by minimizing the (cost) function. 

The LinearRegression class is based on [scipy.linalg.lstsq()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lstsq.html) function which calculates the least squares solution to an equation.

In [ ]:
# Fit a linear model to a set of muon tracks in CMS using linear regression
from sklearn.linear_model import LinearRegression

# The array must be 2D so we will reshape the data before fitting
X = track['x'].values.reshape(-1, 1)
Y = track['y'].values.reshape(-1, 1)

In [ ]:
# Fit a linear model and name it lin_reg

In [ ]:
# Print the model's coefficient and intercept

In [ ]:
# Use the model to make predictions for a new range of values called X_new
X_new = np.array([.25, .5, .75, 1]).reshape(-1, 1)

### Linear Model Fitting using Stochastic Gradient Descent

Gradient Descent is a generic optimization algorithm capable of finding optimal solutions to a wide variety of problems by iteratively changing parameters to minimize a cost function. Gradient Descent measures the local gradient of the error function and goes in the direction of descending gradient. The size of the step is the learning rate. 

We will use SGDRegressor from the sklearn library to implement machine learning techniques to fit the linear model. SGD stands for Stochastic Gradient Descent: picks a random instance in the training set at every step and computes the gradients based only on that single instance. More details about the function can be found [here.](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html)

We will perform Linear Regression using SGD for maximum 1000 epochs (max_iter=1000) or until the loss drops by less than 1e-3 during one epoch (tol=1e-3), starting with a learning rate of 0.3 (eta0=0.3).

In [ ]:
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3, eta0=0.3, random_state=42)
sgd_reg.fit(X,Y.ravel())
sgd_params = sgd_reg.coef_, sgd_reg.intercept_
sgd_params

The function that determines the learning rate at each iteration is called the learning schedule. If the learning rate is reduced too quickly, you may get stuck in a local minimum or even end up frozen before reaching the minimum. If the learning rate is reduced too slowly, you may jump around the minimum for a long time.

In [ ]:
# Decrease the learning rate to 0.1

In [ ]:
# Increase the learning rate to 3

### Polynomial Regression
We know that the path of a muon is not a simple straight line. We can use a linear model to fit nonlinear data by adding features to our variable X in order to use linear regression. We can now fit our data to a parabola by adding a second feature.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X)

In [ ]:
# use the transformed array to make a new linear model and name it poly_reg

In [ ]:
poly_reg_params = poly_reg.coef_, poly_reg.intercept_

# what quantity does this print?
poly_reg_params[1]

In [ ]:
# plot the original data and the optimized models (i.e., trendlines)
plt.scatter(track['x'], track['y'], label="data", color='k')
plt.plot(track['x'], linear_model(track['x'], lin_reg_params[0][0][0],lin_reg_params[1][0]), label="linear regression",color='red')
plt.plot(track['x'], linear_model(track['x'], sgd_params[0],sgd_params[1]), label="SGD",color='blue')
plt.plot(track['x'], (poly_reg_params[0][0][1]*np.square(track['x']) + poly_reg_params[0][0][0]*track['x'] + poly_reg_params[1]), label="polynomial regg model",color='black')

plt.legend()
plt.show()

In [ ]:
# compare the models using chi-sq or RSS

### Regularized Models

So far we have trained models setting its parameters so that the model best fits the training set. If you perform a high-degree polynomial regression, you will fit the training data much better than the Linear Regression. However, fitting to a 300-degree polynomial data will cause the model to try to get as close to the data as possible, overfitting the training data, while the linear model is underfitting the training data. 

To reduce overfitting, we regularize or constain the model. For instance to regularize a polynomial model, you reduce the number of degrees it has. For a linear model, regularization is done by constraining the weights of the model. 

Ridge model solves a regression model where the loss function is the linear least squares function. It is a regularized version of Linear Regression. It adds a regularization term to the cost function. This forces the learning algorithm to keep the weights as small as possible during training. The hyperparameter (alpha) controls how much you want to regulatize the model. 

You can learn more about the model and Ridge Model in the sklearn library [here.](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) 

We will fit the data to a ridge linear model and Sochastic Average Gradient descent (solver="sag") to a second degree polynomial (degree=2).

In [ ]:
from sklearn.linear_model import Ridge
ridge_reg_sag = Ridge(alpha=1, solver="sag")
ridge_reg_sag = Pipeline([
                    ("poly_features", PolynomialFeatures(degree=2, include_bias=False)),
                    ("std_scaler", StandardScaler()),
                    ("regul_reg", ridge_reg_sag),
])
ridge_reg_sag.fit(X,Y)

In [ ]:
# Compare the Ridge Model prediction to your earlier lin_reg prediction

[Least Absolute Shrinkage and Selection Operator (Lasso) Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) is another regularized version of Linear Regression.

In [ ]:
# Compare the Lasso regression prediction to your earlier predictions. Set the regularization constant to 1

## Conclusion  


We have repeated the Muon Tracking exercise to perform a linear fit to the data. We improved our fitting techniques by using polynomial regression. We also introduced two new fitting techniques that use regularization to prevent overfitting our data: the Ridge Model and Lasso Model.